In [18]:
# First XGBoost model for Pima Indians dataset
import numpy as np
import pandas as pd
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
Activities = [1,2,3,4,5,6,7,8,9,10,11]

In [19]:
def getSamplesFromFiles(filesNames=["mHealth_ECGProcessed_subject3.log"],readingsPerSample=50):
    targets  = np.array([])
    feature_set= np.array([])
    for fileName in filesNames:
        df = pd.read_csv(fileName, header=None, delim_whitespace=True)
        for i in range(int(df.shape[0]/readingsPerSample)):
            if df[2][i*readingsPerSample:i*readingsPerSample+readingsPerSample].mean() in Activities:
                feature_set=np.append(feature_set,[df[1][i*readingsPerSample:i*readingsPerSample+readingsPerSample]])
                targets=np.append(targets,df[2][i*readingsPerSample:i*readingsPerSample+readingsPerSample].mean()-1)

    feature_set=feature_set.reshape(int(feature_set.shape[0]/readingsPerSample),readingsPerSample)
#     one_hot_labels=one_hot_labels.reshape(int(one_hot_labels.shape[0]/12),12)
    return feature_set, targets
def unison_shuffled_copies(a, b,seed=1):
    assert len(a) == len(b)
    np.random.seed(seed)
    p = np.random.permutation(len(a))
    return a[p], b[p]

In [20]:
feature_set,targets=getSamplesFromFiles(["mHealth_ECGProcessed_subject3.log","mHealth_ECGProcessed_subject4.log","mHealth_ECGProcessed_subject7.log"],50)
feature_set,targets=unison_shuffled_copies(feature_set,targets)
X_train, X_test, y_train, y_test = train_test_split(feature_set, targets, test_size=0.33, random_state=10)

model = XGBClassifier()
model.fit(X_train, y_train)
# make predictions for test data
y_pred = model.predict(X_test)
predictions = [round(value) for value in y_pred]
# evaluate predictions
accuracy = accuracy_score(y_test, predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

Accuracy: 23.87%


In [31]:
from sklearn.model_selection import GridSearchCV

In [38]:
from sklearn.model_selection import cross_val_score

In [51]:
alg = xgb.XGBClassifier(objective='multi:softprob')

In [41]:
cross_val_score(alg, X_test, y_test, cv=5, scoring='accuracy')

array([0.27737226, 0.13432836, 0.18939394, 0.19847328, 0.2578125 ])

In [44]:
clf = GridSearchCV(alg,{'max_depth': [6,8,10,12],
                        'n_estimators': [50,100,200,400]}, 
                        verbose=1, 
                        error_score='neg_log_loss')

clf.fit(X_train,y_train)
clf.best_score_, clf.best_params_

C:\Users\pcroot\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Fitting 3 folds for each of 16 candidates, totalling 48 fits


[Parallel(n_jobs=1)]: Done  48 out of  48 | elapsed:  7.9min finished


(0.2619047619047619, {'max_depth': 6, 'n_estimators': 200})

In [45]:
y_pred = clf.predict(X_test)
predictions = [round(value) for value in y_pred]
# evaluate predictions
accuracy = accuracy_score(y_test, predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

Accuracy: 26.44%


In [57]:
params={
    'max_depth': [3,4,5,6,7,8,9], # 5 is good but takes too long in kaggle env
    'subsample': [0.4,0.5,0.6,0.7,0.8,0.9,1.0],
    'colsample_bytree':[0.5,0.6,0.7,0.8],
    'n_estimators':[1000,2000,3000],
    'reg_alpha': [0.01, 0.02, 0.03, 0.04]
}

In [ ]:
rs = GridSearchCV(alg,
                  params,
                  cv=5,
                  scoring="neg_log_loss",
                  n_jobs=1,
                  verbose=2)
rs.fit(X_train,y_train)
best_est = rs.best_estimator_
print(best_est)

Fitting 5 folds for each of 2352 candidates, totalling 11760 fits
[CV] colsample_bytree=0.5, max_depth=3, n_estimators=1000, reg_alpha=0.01, subsample=0.4 


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV]  colsample_bytree=0.5, max_depth=3, n_estimators=1000, reg_alpha=0.01, subsample=0.4, total=  17.4s
[CV] colsample_bytree=0.5, max_depth=3, n_estimators=1000, reg_alpha=0.01, subsample=0.4 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   17.9s remaining:    0.0s


[CV]  colsample_bytree=0.5, max_depth=3, n_estimators=1000, reg_alpha=0.01, subsample=0.4, total=  17.5s
[CV] colsample_bytree=0.5, max_depth=3, n_estimators=1000, reg_alpha=0.01, subsample=0.4 
[CV]  colsample_bytree=0.5, max_depth=3, n_estimators=1000, reg_alpha=0.01, subsample=0.4, total=  17.9s
[CV] colsample_bytree=0.5, max_depth=3, n_estimators=1000, reg_alpha=0.01, subsample=0.4 
[CV]  colsample_bytree=0.5, max_depth=3, n_estimators=1000, reg_alpha=0.01, subsample=0.4, total=  17.8s
[CV] colsample_bytree=0.5, max_depth=3, n_estimators=1000, reg_alpha=0.01, subsample=0.4 
[CV]  colsample_bytree=0.5, max_depth=3, n_estimators=1000, reg_alpha=0.01, subsample=0.4, total=  39.6s
[CV] colsample_bytree=0.5, max_depth=3, n_estimators=1000, reg_alpha=0.01, subsample=0.5 
[CV]  colsample_bytree=0.5, max_depth=3, n_estimators=1000, reg_alpha=0.01, subsample=0.5, total=  43.3s
[CV] colsample_bytree=0.5, max_depth=3, n_estimators=1000, reg_alpha=0.01, subsample=0.5 
[CV]  colsample_bytree=0.

...

[CV]  colsample_bytree=0.5, max_depth=6, n_estimators=2000, reg_alpha=0.02, subsample=0.9, total=  36.5s
[CV] colsample_bytree=0.5, max_depth=6, n_estimators=2000, reg_alpha=0.02, subsample=0.9 
[CV]  colsample_bytree=0.5, max_depth=6, n_estimators=2000, reg_alpha=0.02, subsample=0.9, total=  35.9s
[CV] colsample_bytree=0.5, max_depth=6, n_estimators=2000, reg_alpha=0.02, subsample=1.0 
[CV]  colsample_bytree=0.5, max_depth=6, n_estimators=2000, reg_alpha=0.02, subsample=1.0, total=  36.0s
[CV] colsample_bytree=0.5, max_depth=6, n_estimators=2000, reg_alpha=0.02, subsample=1.0 
[CV]  colsample_bytree=0.5, max_depth=6, n_estimators=2000, reg_alpha=0.02, subsample=1.0, total=  36.1s
[CV] colsample_bytree=0.5, max_depth=6, n_estimators=2000, reg_alpha=0.02, subsample=1.0 
[CV]  colsample_bytree=0.5, max_depth=6, n_estimators=2000, reg_alpha=0.02, subsample=1.0, total=  35.9s
[CV] colsample_bytree=0.5, max_depth=6, n_estimators=2000, reg_alpha=0.02, subsample=1.0 
[CV]  colsample_bytree=0.

[CV]  colsample_bytree=0.5, max_depth=6, n_estimators=2000, reg_alpha=0.04, subsample=0.4, total=  31.1s
[CV] colsample_bytree=0.5, max_depth=6, n_estimators=2000, reg_alpha=0.04, subsample=0.4 
[CV]  colsample_bytree=0.5, max_depth=6, n_estimators=2000, reg_alpha=0.04, subsample=0.4, total=  31.6s
[CV] colsample_bytree=0.5, max_depth=6, n_estimators=2000, reg_alpha=0.04, subsample=0.4 
[CV]  colsample_bytree=0.5, max_depth=6, n_estimators=2000, reg_alpha=0.04, subsample=0.4, total=  31.6s
[CV] colsample_bytree=0.5, max_depth=6, n_estimators=2000, reg_alpha=0.04, subsample=0.4 
[CV]  colsample_bytree=0.5, max_depth=6, n_estimators=2000, reg_alpha=0.04, subsample=0.4, total=  31.7s
[CV] colsample_bytree=0.5, max_depth=6, n_estimators=2000, reg_alpha=0.04, subsample=0.5 
[CV]  colsample_bytree=0.5, max_depth=6, n_estimators=2000, reg_alpha=0.04, subsample=0.5, total=  33.3s
[CV] colsample_bytree=0.5, max_depth=6, n_estimators=2000, reg_alpha=0.04, subsample=0.5 
[CV]  colsample_bytree=0.

[CV]  colsample_bytree=0.5, max_depth=6, n_estimators=3000, reg_alpha=0.01, subsample=0.5, total=  45.2s
[CV] colsample_bytree=0.5, max_depth=6, n_estimators=3000, reg_alpha=0.01, subsample=0.6 
[CV]  colsample_bytree=0.5, max_depth=6, n_estimators=3000, reg_alpha=0.01, subsample=0.6, total=  46.2s
[CV] colsample_bytree=0.5, max_depth=6, n_estimators=3000, reg_alpha=0.01, subsample=0.6 
[CV]  colsample_bytree=0.5, max_depth=6, n_estimators=3000, reg_alpha=0.01, subsample=0.6, total=  46.2s
[CV] colsample_bytree=0.5, max_depth=6, n_estimators=3000, reg_alpha=0.01, subsample=0.6 
[CV]  colsample_bytree=0.5, max_depth=6, n_estimators=3000, reg_alpha=0.01, subsample=0.6, total=  46.3s
[CV] colsample_bytree=0.5, max_depth=6, n_estimators=3000, reg_alpha=0.01, subsample=0.6 
[CV]  colsample_bytree=0.5, max_depth=6, n_estimators=3000, reg_alpha=0.01, subsample=0.6, total=  46.5s
[CV] colsample_bytree=0.5, max_depth=6, n_estimators=3000, reg_alpha=0.01, subsample=0.6 
[CV]  colsample_bytree=0.

[CV]  colsample_bytree=0.5, max_depth=6, n_estimators=3000, reg_alpha=0.02, subsample=0.7, total=  47.1s
[CV] colsample_bytree=0.5, max_depth=6, n_estimators=3000, reg_alpha=0.02, subsample=0.7 
[CV]  colsample_bytree=0.5, max_depth=6, n_estimators=3000, reg_alpha=0.02, subsample=0.7, total=  47.7s
[CV] colsample_bytree=0.5, max_depth=6, n_estimators=3000, reg_alpha=0.02, subsample=0.7 
[CV]  colsample_bytree=0.5, max_depth=6, n_estimators=3000, reg_alpha=0.02, subsample=0.7, total=  47.1s
[CV] colsample_bytree=0.5, max_depth=6, n_estimators=3000, reg_alpha=0.02, subsample=0.8 
[CV]  colsample_bytree=0.5, max_depth=6, n_estimators=3000, reg_alpha=0.02, subsample=0.8, total=  47.5s
[CV] colsample_bytree=0.5, max_depth=6, n_estimators=3000, reg_alpha=0.02, subsample=0.8 
[CV]  colsample_bytree=0.5, max_depth=6, n_estimators=3000, reg_alpha=0.02, subsample=0.8, total=  47.5s
[CV] colsample_bytree=0.5, max_depth=6, n_estimators=3000, reg_alpha=0.02, subsample=0.8 
[CV]  colsample_bytree=0.

[CV]  colsample_bytree=0.5, max_depth=6, n_estimators=3000, reg_alpha=0.03, subsample=0.9, total=  48.7s
[CV] colsample_bytree=0.5, max_depth=6, n_estimators=3000, reg_alpha=0.03, subsample=0.9 
[CV]  colsample_bytree=0.5, max_depth=6, n_estimators=3000, reg_alpha=0.03, subsample=0.9, total=  48.9s
[CV] colsample_bytree=0.5, max_depth=6, n_estimators=3000, reg_alpha=0.03, subsample=0.9 
[CV]  colsample_bytree=0.5, max_depth=6, n_estimators=3000, reg_alpha=0.03, subsample=0.9, total=  48.7s
[CV] colsample_bytree=0.5, max_depth=6, n_estimators=3000, reg_alpha=0.03, subsample=0.9 
[CV]  colsample_bytree=0.5, max_depth=6, n_estimators=3000, reg_alpha=0.03, subsample=0.9, total=  51.3s
[CV] colsample_bytree=0.5, max_depth=6, n_estimators=3000, reg_alpha=0.03, subsample=0.9 
[CV]  colsample_bytree=0.5, max_depth=6, n_estimators=3000, reg_alpha=0.03, subsample=0.9, total=  49.8s
[CV] colsample_bytree=0.5, max_depth=6, n_estimators=3000, reg_alpha=0.03, subsample=1.0 
[CV]  colsample_bytree=0.

In [ ]:
y_pred = rs.predict(X_test)
predictions = [round(value) for value in y_pred]
# evaluate predictions
accuracy = accuracy_score(y_test, predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))